In [1]:
import pandas as pd
import numpy as np
import joblib
from keras.models import Sequential, load_model
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

dataset = pd.read_csv("heston_calib2.csv")
#dataset = dataset.dropna()
#dataset[dataset < 0] = 0


$X$ deve corrispondere agli input su cui hai fatto il training **(nello stesso ordine esatto !)**
Implicitamente hai definito 32 opzioni perchè nel loop con cui hai calcolato questi valori hai usato 4 valori della time to maturity (0.5, 1, 1.5, 2) e 8 valori di $K$ (85, 115, 5). Quindi hai 33 input perchè hai 32 prezzi delle opzioni devi aggiungere $S_t$, $r$ essendo fissato non viene considerato

In questo caso gli output non ti servono perchè vuoi una predizione della NN dovrai semmai confrontare alla fine i valori ottenuti con quelli usati per generare i prezzi in $X$.

In [2]:
X = dataset.iloc[:, :33] 
#y = dataset.iloc[:, 33:].to_numpy() 

Esatto questa è la trasformazione, ma così facendo ne crei una nuova, invece devi applicare quella che è stata definita al momento del training perchè il sample deve essere nelle stesse condizioni per avere risultati sensati.
La trasformazione semplicemente riporta i valorti di input tra 0 e 1 (quindi il massimo valore del sample corrisponderà a 1 il minimo a 0 e quelli intermedi saranno riscalati nel range[0,1]) Immagina di aver usato per il training tre soli valori:

90, 100 e 110

la trasformazione è banalmente $x' = \cfrac{(x-90)}{(110-90)}$, quindi

90-> 0, 100->0.5 e 110->1

Ora considera di usare due valori di market quotes

95 e 105

Se tu creassi una nuova trasformazione 95->0 e 105->1 $\left(x" = \cfrac{(x-95)}{(105-95)}\right)$ mentre applicando quella usata nel training *(cioè la cosa giusta da fare)*

95->0.25 e 105->0.75

Quindi come vedi in un caso calcoli il tranining con input (0, 1), nell'altro (0.25, 0.75) e naturalmente la NN darà risultati diversi.

In [3]:
#scale_x = MinMaxScaler().fit(X)
#X = scale_x.transform(X)

#scale_y = MinMaxScaler().fit(y)
#y = scale_y.transform(y)

# load scaling transformations
scale_x = joblib.load("heston_x.save")
scale_y = joblib.load("heston_y.save")

# qui applica la trasformazione
X = scale_x.transform(X)

# questo è solo per motivi tecnici trasformi le X da riga a colonna...
inp = X.T

In [4]:
# carichi il modello salvato con il training 
model = load_model('prova_relu.h5')

In [5]:
# p contiene le predizioni della NN per le market quotes
p = model.predict(X)

$p$ è un vettore i cui elementi sono quintuplette corrispondenti ai valori dei parametri del modello di Heston per le quali vale lo stesso discorso fatto per $X$, ovvero abbiamo fatto il training in modo tale che ritornasse i parametri normalizzati tra 0-1 quindi bisogna ora applicare la trasformazione inversa ($scale\_y$).

In [6]:
r = scale_y.inverse_transform(p)
print ("Calibrated parameters")
print (r)

Calibrated parameters
[[ 4.35242701e+00  6.92202884e-04  6.84461221e-02  5.72631240e-01
   1.00788839e-01]
 [ 4.41849661e+00  6.92802365e-04  7.40465894e-02  4.57714170e-01
   9.28783268e-02]
 [ 4.44528294e+00  7.43458397e-04  7.04838261e-02  5.44153214e-01
   9.91369188e-02]
 [ 4.30928898e+00  6.69110217e-04  6.76826835e-02  5.81593096e-01
   1.01191632e-01]
 [ 4.41655159e+00  6.89726439e-04  7.42131919e-02  4.53073621e-01
   9.25581306e-02]
 [ 4.40396738e+00  6.68115565e-04  7.54433125e-02  4.16361690e-01
   9.00436193e-02]
 [ 4.30199766e+00  5.28275850e-04  8.04398209e-02  3.28683078e-01
   8.07998627e-02]
 [ 4.37727928e+00  6.23152067e-04  7.73779154e-02  3.85335416e-01
   8.63322020e-02]
 [ 4.19360924e+00  3.97027907e-04  8.49691555e-02  2.22100675e-01
   7.11158141e-02]
 [ 4.20530701e+00  1.52442793e-04  1.13862179e-01 -1.01735377e+00
   3.15072984e-02]
 [ 4.84739971e+00  1.55843896e-04  1.18810728e-01 -1.78323638e+00
   2.51270551e-02]
 [ 3.69307637e+00  4.36030852e-04  1.548263

Fatto questo riprezzi le opzioni con i parametri di Heston calibrati (anche in questo caso dovresti trovare valori dei prezzi vicini alle market quotes).
